In [ ]:
import boost_histogram as bh
import numpy as np
import matplotlib.pyplot as plt
import hist
from hist import Hist

In [ ]:
def make_2D_data(*, mean=(0,0), widths=(1,1), size=1_000_000):
    cov = np.asarray(widths) * np.eye(2)
    return np.random.multivariate_normal(mean, cov, size=size).T

def plothist2d(h):
    X, Y = h.axes.edges

    return plt.pcolormesh(X.T, Y.T, h.view().T)

plothist = lambda h: plt.bar(h.axes[0].centers, h, width=h.axes[0].widths)

# 4: Special histogram axes

## 4.1: Transforms

There are more axes types, and they all provide the same API in histograms, so they all just work without changes:

In [ ]:
hist8 = bh.Histogram(
    bh.axis.Regular(30, 1,10, transform=bh.axis.transform.log),
    bh.axis.Regular(30, 1,10, transform=bh.axis.transform.sqrt)
)

In [ ]:
hist8.reset()
hist8.fill(*make_2D_data(mean=(5,5), widths=(5,5)))

In [ ]:
plothist2d(hist8);

## 4.2: And, circular, too!

In [ ]:
hist9 = bh.Histogram(bh.axis.Regular(30, 0, 2*np.pi, circular=True))
hist9.fill(np.random.uniform(0, np.pi*4, size=300))

Now, the really complicated part, making a circular histogram:


In [ ]:
ax = plt.subplot(111, polar=True)
plothist(hist9);

Not bad, we just reused our plotting code for all 1D histograms!

## 4.3: Axis Types

Based on boost-histogram’s Axis, hist support six types of axis, `Regular`, `Boolean`, `Variable`, `Integer`, `IntCategory` and `StrCategory` with additional names and labels.

Names are pretty useful for some histogramming shortcuts, thus greatly facilitate HEP’s studies. Note that the name is the identifier for an axis in a histogram and must be unique.

In [ ]:
axis0 = hist.axis.Regular(10, -5, 5, overflow=False, underflow=False, name="A")
axis1 = hist.axis.Boolean(name="B")
axis2 = hist.axis.Variable(range(10), name="C")
axis3 = hist.axis.Integer(-5, 5, overflow=False, underflow=False, name="D")
axis4 = hist.axis.IntCategory(range(10), name="E")
axis5 = hist.axis.StrCategory(["T", "F"], name="F")

Histogram is consisted with various axes, there are two ways to create a histogram, currently. You can either fill a histogram object with axes or add axes to a histogram object. You cannot add axes to an existing histogram. _Note that to distinguish these two method, the second way has different axis type names (abbr.)._

In [ ]:
# fill the axes
h = Hist(axis0, axis1, axis2, axis3, axis4, axis5)

In [ ]:
# add the axes using the shortcut method
h = (
    Hist.new.Reg(10, -5, 5, overflow=False, underflow=False, name="A")
    .Bool(name="B")
    .Var(range(10), name="C")
    .Int(-5, 5, overflow=False, underflow=False, name="D")
    .IntCat(range(10), name="E")
    .StrCat(["T", "F"], name="F")
    .Double()
)

In [ ]:
assert h.axes[0].name == axis0.name
assert h.axes[1].label == axis1.name  # label will be returned as name if not provided
assert all(h.axes[2].widths == axis2.widths)
assert all(h.axes[3].edges == axis3.edges)
assert h.axes[4].metadata == axis4.metadata
assert all(h.axes[5].centers == axis5.centers)